In [1]:
import numpy as np
import random
import time

from sklearn.linear_model import LinearRegression


### combine RGB and std into feature for each color 

In [2]:
r1,g1,b1, r2,g2,b2, r3,g3,b3, r4,g4,b4 =np.loadtxt('train_features.dat',unpack=True)

rgb1 = np.vstack((r1,g1,b1)).T
rgb2 = np.vstack((r2,g2,b2)).T
rgb3 = np.vstack((r3,g3,b3)).T 
rgb4 = np.vstack((r4,g4,b4)).T

std1 = [np.std(color) for color in rgb1]
std2 = [np.std(color) for color in rgb2]
std3 = [np.std(color) for color in rgb3]
std4 = [np.std(color) for color in rgb4]

x1 = np.vstack( (rgb1.T, std1) ).T
x2 = np.vstack( (rgb2.T, std2) ).T
x3 = np.vstack( (rgb3.T, std3) ).T
x4 = np.vstack( (rgb4.T, std4) ).T



### Separate color palette into X and y

1. Pick 3 out of 4 colors to use for training model-- do this twice for each palette

2. Pick 2 out of 3 as X, other color is Y--do twice for each palette

3. Interchange order of X colors (Y should not depend on the order of input colors)

Gives 8 rows of X/y for each palette

In [3]:
def pick_three_from_palette( c1,c2,c3,c4):
    # c1,...,c4 are one row from x1,...,x4 arrays (where each row c is r,g,b,std of a different image)
    (exclude1, exclude2) = random.sample([0,1,2,3],2)
    colors = [c1,c2,c3,c4]
    set1 = np.array( [list(colors[i]) for i in range(4) if i!=exclude1 ])
    set2 = np.array( [list(colors[i]) for i in range(4) if i!=exclude2 ])
    return( set1, set2)

def pick_two_from_three(setofthree):
    (exclude1, exclude2) = random.sample([0,1,2],2)
    X1 = [list(setofthree[i]) for i in range(3) if i!=exclude1 ]
    y1 = setofthree[exclude1]
    X2 = [list(setofthree[i]) for i in range(3) if i!=exclude2 ]
    y2 = setofthree[exclude2]
    return (np.array(X1), np.array(X2), y1, y2)

def interchange_x_colors(X):
    X_reversed = np.array(list(reversed(X)))
    return (X.reshape(8) , X_reversed.reshape(8)) 

In [216]:
full_X_list=[]
full_y_list=[]

for i in range(len(x1)):

    set1, set2 = pick_three_from_palette(x1[i], x2[i], x3[i], x4[i])
    for set in [set1, set2]:
        X1, X2, y1, y2 = pick_two_from_three(set)

        for (x,y) in [(X1,y1), (X2,y2)]:
            #print(y)
            Xa, Xb = interchange_x_colors(x)
            full_X_list.append(Xa)
            full_y_list.append(y)
            full_X_list.append(Xb)
            full_y_list.append(y)

(y_R, y_G, y_B, y_std) = np.array(full_y_list).T

In [220]:
full_X_list=[]
full_y_list=[]

for i in range(len(x1)):
    set1, set2 = pick_three_from_palette(x1[i], x2[i], x3[i], x4[i])
    for set in [set1, set2]:
        X1, X2, y1, y2 = pick_two_from_three(set)
        full_X_list.append(X1.reshape(8))
        full_y_list.append(y1)
        full_X_list.append(X2.reshape(8))
        full_y_list.append(y2)
                

(y_R, y_G, y_B, y_std) = np.array(full_y_list).T

### Fit model for each prediction feature (R/G/B/stdev)

In [187]:
from sklearn.linear_model import LinearRegression

regression_model_R = LinearRegression(normalize=True)
regression_model_G = LinearRegression(normalize=True)
regression_model_B = LinearRegression(normalize=True)
regression_model_std = LinearRegression(normalize=True)


t0=time.time()
regression_model_R.fit(full_X_list, y_R)
regression_model_G.fit(full_X_list, y_G)
regression_model_B.fit(full_X_list, y_B)
regression_model_std.fit(full_X_list, y_std)
print(time.time() - t0)

0.45084071159362793


In [186]:
regression_model_R.coef_

array([ 0.03400273, -0.06812489, -0.14319724, -0.0327027 ,  0.03400273,
       -0.06812489, -0.14319724, -0.0327027 ])

In [188]:
regression_model_G.coef_

array([-0.15847685,  0.15007159, -0.18760174, -0.00973323, -0.15847685,
        0.15007159, -0.18760174, -0.00973323])

In [189]:
regression_model_B.coef_

array([-0.10962093, -0.08962852,  0.05681749, -0.04309547, -0.10962093,
       -0.08962852,  0.05681749, -0.04309547])

In [190]:
regression_model_std.coef_

array([ 0.0103689 ,  0.00233172, -0.010001  ,  0.10208932,  0.0103689 ,
        0.00233172, -0.010001  ,  0.10208932])

In [316]:
from importlib import reload

import ColorModel
reload(ColorModel)
import ColorModel

In [317]:
train_data = ColorModel.color_dataset('train_features.dat')

In [318]:
train_data.convert_data_to_features()

In [319]:
train_data.linear_add_features()
train_data.quadratic_add_features()